# Preparing ARIA Sentinel-1 data for validation of Solid Earth requirements

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun  <br>
Extensive modifications by Adrian Borsa and Amy Whetter 2022 <br>
Reorganized and modified by Ekaterina Tymofyeyeva, March 2024

<div class="alert alert-warning">
This notebook pre-processes data for different NISAR Solid Earth calval sites amd requirements. Subsequent validation is done via separate notebooks for the Transient, Secular, and Coseismic requirements. These are located under /ATBD_main/methods/.
</div>

<hr/>

## Table of Contents: <a id='prep_TOC'></a>

[**Environment Setup**](#setup)
- [Load Python Packages](#load_packages)
- [Define CalVal Site and Parameters](#set_calval_params)
- [Define Directories](#set_directories)
- [Authentication](#set_authentication)

[**1. Download and Prepare Interferograms**](#prep_ifg)
- [1.1.  Download Interferograms](#prep_download_ifg)
- [1.2.  Crop Interferograms](#prep_crop_ifg)
- [1.3.  Set Up MintPy Configuration file](#prep_setup_config)
- [1.4.  Load Data into MintPy](#prep_load_data)

<hr/>

<a id='#setup'></a>
## Environment Setup

### Load Python Packages <a id='#load_packages'></a>

In [ ]:
import glob
import json
import netrc
import numpy as np
import os
import pandas as pd
import subprocess

from datetime import datetime as dt
from matplotlib import pyplot as plt
from mintpy.cli import view, plot_network
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis
from mintpy.utils import ptime, readfile, utils as ut
from pathlib import Path
from scipy import signal
from solid_utils.sampling import load_geo, samp_pair, profile_samples, haversine_distance

# Set gobal plot parameters
plt.rcParams.update({'font.size': 12})

### Define Calval Site and Parameters <a id='set_calval_params'></a>

In [ ]:
# Specify a calval location ID from my_sites.txt
site = 'test' 

# Choose the requirement to validate
# Options: 'Secular' 'Coseismic' 'Transient'
requirement = 'Secular' 

# What dataset are you processing?
#'ARIA_S1' (old directory structure for Sentinel-1 testing with aria-tools)
#'ARIA_S1_new' (new directory structure for Sentinel-1 testing with aria-tools)
dataset = 'ARIA_S1_new'
aria_gunw_version = '3_0_1'

# The date and version of this Cal/Val run
rundate = '20250214'
version = '1'

# Provide the file where you keep your customized list of sites.
custom_sites = '/home/jovyan/my_sites.txt'

# Enter a username for storing your outputs
if os.path.exists('/home/jovyan/me.txt'):
    with open('/home/jovyan/me.txt') as m:
        you = m.readline().strip()
else:
    you = input('Please type a username for your calval outputs:')
    with open ('/home/jovyan/me.txt', 'w') as m: 
        m.write(you)

# Load metadata for calval locations
with open(custom_sites,'r') as fid:
    sitedata = json.load(fid)
sitedata['sites'][site]

### Set Directories and Files <a id='set_directories'></a>

In [ ]:
# Directory location for Cal/Val data (do not change)
start_directory = '/scratch/nisar-st-calval-solidearth' 

# Site directory
site_dir = os.path.join(start_directory, dataset, site)

# Working directory for calval processing
work_dir = os.path.join(site_dir, requirement, you, rundate, 'v' + version)
os.makedirs(work_dir, exist_ok=True)
os.chdir(work_dir)
print("  Work directory:", work_dir)

# Directory for storing GUNW interferograms
gunw_dir = os.path.join(site_dir,'products')
os.makedirs(gunw_dir, exist_ok=True)
print("  GUNW directory:", gunw_dir) 

# Directory for storing MintPy outputs
mintpy_dir = os.path.join(work_dir,'MintPy')
os.makedirs(mintpy_dir, exist_ok=True)
print("MintPy directory:", mintpy_dir)

# Configuration file
config_file = Path(mintpy_dir)/(sitedata['sites'][site]['calval_location'] + '.cfg')

### Authentication <a id='set_authentication'></a>

In [ ]:
# Earthdata login to download GUNWs
fnetrc = '/home/jovyan/.netrc'
earthdata = False
if os.path.exists(fnetrc):
    os.system('chmod 0600 ' + fnetrc)
    remoteHostName  = "urs.earthdata.nasa.gov"
    netrc = netrc.netrc()
    with open(fnetrc) as file:
        if remoteHostName in file.read():
            authTokens = netrc.authenticators(remoteHostName)
            earthdata_user = authTokens[0]
            earthdata_password = authTokens[2]
            earthdata = True          
if not earthdata:             
    print('NEEDED to Download ARIA GUNWs: \n Link to create account : https://urs.earthdata.nasa.gov/')
    earthdata_user = input('Please type your Earthdata username:')
    earthdata_password = input('Please type your Earthdata password:')
    with open(fnetrc, 'a') as file:
        file.write('machine urs.earthdata.nasa.gov\n')
        file.write('login ' + earthdata_user + '\n')
        file.write('password ' + earthdata_password)
        os.system('chmod 0600 ' + fnetrc)

# OpenTopography login to download DEMs
fopentopo = '/home/jovyan/.topoapi'
if os.path.exists(fopentopo):
    os.system('chmod 0600 ' + fopentopo)
    with open(fopentopo) as file:
        opentopography_api_key = file.read()
else:   
    print('NEEDED To Download DEMs: \n Link to get API Key : https://portal.opentopography.org/login' + 
     '\n Goto: My Account > myOpenTopo Authorizations and API Key > Request API key')
    opentopography_api_key = input('Please type your OpenTopo API key:')
    with open(fopentopo, 'a') as file:
        file.write(opentopography_api_key)
        os.system('chmod 0600 ' + fopentopo)

<br>
<hr>

<a id='prep_ifg'></a>
## 1. Download and Prepare Interferograms

In this initial processing step, all the necessary Level-2 unwrapped interferogram products are gathered, organized and reduced to a common grid for analysis with MintPy. Ascending and descending stacks of nearest-neighbor and skip-1 interferograms will be prepared for independent analysis. We use the open-source ARIA-tools package to download processed L2 interferograms over selected cal/val regions from the Alaska Satellite Facility archive and to stitch/crop the frame-based NISAR GUNW products to stacks that can be directly ingested into MintPy for time-series processing. ARIA-tools uses a phase-minimization approach in the product overlap region to stitch the unwrapped and ionospheric phase, a mosaicing approach for coherence and amplitude, and extracts the geometric information from the 3D data cubes through a mosaicking of the 3D datacubes and subsequent intersection with a DEM.

REFERENCE: https://github.com/aria-tools/ARIA-tools

### 1.1. Download GUNW Interferograms <a id='prep_download_ifg'></a>

In [ ]:
print('CalVal site: {}'.format(site))
bbox = sitedata['sites'][site]['download_region']
startdate = sitedata['sites'][site]['download_start_date']
enddate = sitedata['sites'][site]['download_end_date']
track = sitedata['sites'][site]['sentinel_track']

# Make general ARIA_download command
aria_download = 'ariaDownload.py --num_threads 8 -b {bbox} -u {user} -p {password} \
    -s {start}  -e {end} -t {track} --workdir {workdir} --version {version} -o {output}'

# Use ARIA_download to count number of archived GUNW files
command = aria_download.format(bbox = bbox, start = startdate, end = enddate,
                                track = track, version = aria_gunw_version, 
                                user = earthdata_user, password = earthdata_password, 
                                workdir = gunw_dir, output = 'count')
process = subprocess.run(command, text = True, shell = True, \
    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(process.stderr.split('\n')[-2])

# Use ARIA_download to generate list of GUNW URLs 
command = aria_download.format(bbox = bbox, start = startdate, end = enddate,
                                track = track, version = aria_gunw_version, 
                                user = earthdata_user, password = earthdata_password, 
                                workdir = gunw_dir, output = 'Url')
process = subprocess.run(command, text = True, shell = True, \
    stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Use ARIA_download to download GUNWs
print('Starting GUNW download....')
command = aria_download.format(bbox = bbox, start = startdate, end = enddate,
                                track = track, version = aria_gunw_version, 
                                user = earthdata_user, password = earthdata_password, 
                                workdir = gunw_dir, output = 'Download')
process = subprocess.run(command, text = True, shell = True, \
    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print('Finished GUNW download')

# Cleaning unnecessary files
data_to_clean = ["avg_rates.csv", "ASFDataDload0.py", "AvgDlSpeed.png", "error.log"]
for i, file in enumerate(data_to_clean):
    if os.path.exists(os.path.join(gunw_dir,file)):
        print('Cleaning file {}'.format(file))
        os.unlink(os.path.join(gunw_dir,file))
        os.unlink(os.path.join(work_dir,file))

### 1.2. Crop and Mask Interferograms <a id='prep_crop_ifg'></a>

In [ ]:
# Identify subset of available GUNW files within user-specified period
startdate = int(sitedata['sites'][site]['download_start_date'])
enddate = int(sitedata['sites'][site]['download_end_date'])
gunw_list = []
for file in os.listdir(gunw_dir):
    if file.endswith('.nc') and file.find(aria_gunw_version) != -1:
        date1 = file[30:38]
        date2 = file[21:29]
        if int(date1) >= startdate and int(date2) <= enddate:
            gunw_list.append(os.path.join(gunw_dir, file))

# Write GUNW list to file to pass to ariaTSsetup.py
gunw_list.sort()
product_file = f"{work_dir}/product_file.txt"
with open(product_file, "w") as prod_file:
    prod_file.write("\n".join(map(str, gunw_list)))

# Crop interferograms to analysis region and optionally mask data with GSHHS water mask
os.chdir(work_dir)
mask_file = 'auto'
if not os.path.exists(os.path.join(work_dir,'stack')):
    if not os.path.exists('/home/jovyan/.topoapi'): # if OpenTopo API key not already installed
        os.system('echo "{api_key}" > /home/jovyan/.topoapi; \
            chmod 600 /home/jovyan/.topoapi'.format(api_key = str(opentopography_api_key)))
    print('Preparing GUNWs for MintPY....')
    if sitedata['sites'][site]['maskWater'] != 'False':
        command = 'ariaTSsetup.py -f ' + product_file + ' -b ' + \
            sitedata['sites'][site]['analysis_region'] + \
            ' --mask Download  --croptounion -nt 8 --verbose'
        mask_file = '../mask/watermask.msk'
    else: # skip slow mask download when we don't need to mask water
        command = 'ariaTSsetup.py -f ' + product_file + ' -b ' + \
            sitedata['sites'][site]['analysis_region'] + ' --croptounion -nt 8 --verbose'
    # Crop and prepare stack
    process = subprocess.run(command, text = True, shell = True, \
        stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print('Finished preparing GUNWs for MintPy!!')
else:
    print('Stack directory detected and not overwritten.')

### 1.3. Set Up MintPy Configuration file <a id='prep_setup_config'></a>

The default processing parameters for MintPy's **smallbaselineApp.py** need to be modified by including the following lines in config_file (which must be manually created and placed into mint_dir):

- mintpy.load.processor      = aria
- mintpy.load.unwFile        = ../stack/unwrapStack.vrt
- mintpy.load.corFile        = ../stack/cohStack.vrt
- mintpy.load.connCompFile   = ../stack/connCompStack.vrt
- mintpy.load.demFile        = ../DEM/SRTM_3arcsec.dem
- mintpy.load.incAngleFile   = ../incidenceAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.azAngleFile    = ../azimuthAngle/{download_start_date}_{download_edn_date}.vrt
- mintpy.load.waterMaskFile  = ../mask/watermask.msk
- mintpy.reference.lalo      = auto, or somewhere in your bounding box
- mintpy.topographicResidual.pixelwiseGeometry = no
- mintpy.troposphericDelay.method              = no
- mintpy.topographicResidual                   = no

In [ ]:
os.chdir(mintpy_dir)

# Write smallbaseline.py config file
config_file_content = """
mintpy.load.processor = aria
mintpy.compute.numWorker = auto
mintpy.load.unwFile = {wd}/stack/unwrapStack.vrt
mintpy.load.corFile = {wd}/stack/cohStack.vrt
mintpy.load.connCompFile = {wd}/stack/connCompStack.vrt
mintpy.load.demFile = {wd}/DEM/glo_90.dem
mintpy.load.incAngleFile = {wd}/incidenceAngle/*.vrt
mintpy.load.azAngleFile = {wd}/azimuthAngle/*.vrt
mintpy.load.waterMaskFile = {mask_file}
mintpy.topographicResidual.pixelwiseGeometry = no
mintpy.troposphericDelay.method = no
mintpy.topographicResidual = no
mintpy.network.tempBaseMax = {tempmax}
mintpy.network.startDate = {startdatenet}
mintpy.network.endDate = {enddatenet}
mintpy.velocity.startDate = {startdatevel}
mintpy.velocity.endDate = {enddatevel}
mintpy.reference.lalo = {reference_lalo}
mintpy.network.excludeIfgIndex = {excludeIfg}""".format(wd = work_dir,
                                mask_file = mask_file,
                                tempmax=sitedata['sites'][site]['tempBaseMax'],
                                excludeIfg=sitedata['sites'][site]['ifgExcludeList'],
                                startdatenet=sitedata['sites'][site]['download_start_date'],
                                enddatenet=sitedata['sites'][site]['download_end_date'],
                                startdatevel=sitedata['sites'][site]['download_start_date'],
                                enddatevel=sitedata['sites'][site]['download_end_date'],
                                reference_lalo=sitedata['sites'][site]['reference_lalo'])

config_file.write_text(config_file_content)
print('MintPy config file:\n    {}'.format(config_file))
print(config_file.read_text())

### 1.4. Load Data into MintPy Cubes <a id='prep_load_data'></a>

The output of this step is an "inputs" directory in 'calval_directory/calval_location/MintPy/" containing two HDF5 files:
- ifgramStack.h5: This file contains 6 dataset cubes (e.g. unwrapped phase, coherence, connected components etc.) and multiple metadata
- geometryGeo.h5: This file contains geometrical datasets (e.g., incidence/azimuth angle, masks, etc.)

In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep load_data'
process = subprocess.run(command, shell=True)
print('Mintpy input files:')
[x for x in os.listdir('inputs') if x.endswith('.h5')]